## Continu Inzicht Demo (Real time)
> https://tscontinuinzicht.hkvservices.nl/ci-demo

### Deze notebook wordt gebruikt voor het vullen van Continu Inzicht Demo database
- tijdstippen van de momenten bijwerken in de momenten tabel
- waterstanden van de meetstations opslaan in de data tabel
- van alle waterstanden de waterstand voor de momenten halen >> momenten: [-24,0,24,48]
- classificeren van waterstanden per meetstation en opslaan in de statustabel
- bepalen van de maatgegevende waterstand per dijkvak

**Imports**

In [ ]:
# import algemeen
from pathlib import Path

# imports base modules
from toolbox_continu_inzicht import Config
from toolbox_continu_inzicht import DataAdapter

**Inlezen configuratie**
<details>
<summary>Configuratie bekijken</summary>   

```text
    GlobalVariables:
        rootdir: "data"     
        calc_time: "2024-11-19"
        moments: [-24,0,24,48]     

        LoadsWaterinfo:    
                parameters: ["waterhoogte"]
                MISSING_VALUE: 999

        LoadsMatroos:
            website: "noos"
            model: "fews_rmm_km"
            parameters: ["WATHTE"]              

        LoadsFews:
            host: "https://fews.hhnk.nl"
            port: 443
            region: "fewspiservice"
            version: "1.25"
            filter: ""
            parameters: [ "WATHTE [m][NAP][OW]" ]    
            uurdata: true,
            addnearestmoments: true,
            addmaxmoment: false       

        LoadsFewsLimburg:
            host: "http://109.232.44.231"
            port: 443
            region: "fewspiservice"
            version: "1.25"
            filter: "App_Maas"
            parameters: [ "WATHTE" ]    
            uurdata: true,
            addnearestmoments: true,
            addmaxmoment: false           

        LoadsWaterwebservicesRWS:        
            parameters: ["WATHTE"]
            MISSING_VALUE: -999           

        DataAdapter: 

            default_options:
                csv:
                    sep: ","

            in_measuringstation_waterinfo: 
                type: ci_postgresql_from_measuringstations
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"
                source: "waterinfo"        

            in_measuringstation_matroos: 
                type: ci_postgresql_from_measuringstations
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"
                source: "matroos"

            in_measuringstation_fews_hhnk:
                type: ci_postgresql_from_measuringstations
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"
                source: "fews_hhnk"   

            in_measuringstation_fews_limburg:          
                type: ci_postgresql_from_measuringstations
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"
                source: "fews_limburg" 

            out_waterstanden_ci_postgresql_001: 
                type: ci_postgresql_to_data
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"
                objecttype: "measuringstation"     
                unit_conversion_factor: 0.001  

            out_waterstanden_ci_postgresql_1: 
                type: ci_postgresql_to_data
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"
                objecttype: "measuringstation"     
                unit_conversion_factor: 1           

            in_measuringstation_data_table: 
                type: ci_postgresql_measuringstation_data_table
                database: geoserver        
                schema: continuinzicht_demo_realtime    

            in_measuringstation_conditions_table: 
                type: ci_postgresql_from_conditions
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"   

            out_measuringstation_states_table:   
                type: ci_postgresql_to_states
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"  
                objecttype: "measuringstation"                    

            in_section_table: 
                type: ci_postgresql_from_sections
                database: "geoserver"
                schema: "continuinzicht_demo_realtime" 

            in_sectionfractions_table: 
                type: ci_postgresql_from_sectionfractions
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"           

            out_moments_table:   
                type: ci_postgresql_to_moments
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"  

            out_waterstanden_section_ci_postgresql: 
                type: ci_postgresql_to_data
                database: "geoserver"
                schema: "continuinzicht_demo_realtime"
                objecttype: "section"             

            df_moment_waterstanden: 
                type: python

```
</details>

In [ ]:
# Lees configuratiebestand
yaml_config_file = "continu-inzicht-demo.yaml"
data_path = Path.joinpath(Path.cwd(), "data", yaml_config_file)

config = Config(config_path=data_path)
config.lees_config()

**Aanmaken adapter**

In [ ]:
# data adapter aanmaken
data_adapter = DataAdapter(config=config)

**Start Continu Inzicht berekening**

In [ ]:
from toolbox_continu_inzicht.helpers import calculation_start

start, end = calculation_start(
    data_adapter=data_adapter, output="calculation_start_config", calc_time=None
)
print(f"{start} - {end}")

**Ophalen waterstanden via Waterinfo**

In [ ]:
# Waterinfo waterstanden
# Vanuit WRIJ: vak 48-1_5, waarschijnlijk 100% gekoppeld aan Lobith – data uit RWS Waterinfo
# https://waterinfo.rws.nl/api/chart/get?mapType=waterhoogte&locationCodes=Lobith(LOBI)&values=-48%2C48

from toolbox_continu_inzicht.loads import LoadsWaterinfo

waterinfo = LoadsWaterinfo(data_adapter=data_adapter)
waterinfo.run(
    input="in_measuringstation_waterinfo", output="out_waterstanden_ci_postgresql_001"
)
df_out = waterinfo.df_out

**Ophalen waterstanden via Noos Matroos**

In [ ]:
# Noos Matroos waterstanden
# Vanuit WSBD: vak 34-2_31, waarschijnlijk gekoppeld aan 2 stations vanuit NOOS Matroos (Hollandsch Diep km 993 en Hollandsch Diep km km 993)
# https://noos.matroos.rws.nl/direct/get_series.php?source=fews_rmm_km&unit=waterlevel&loc=Hollandsch%20Diep%20km%20994&anal_time=0000-00-00T00%3A00%3A00Z&tstart=2024-11-18T00%3A00%3A00.000Z&tstop=2024-11-22T00%3A00%3A00.000Z&format=dd
from toolbox_continu_inzicht.loads import LoadsMatroos

matroos = LoadsMatroos(data_adapter=data_adapter)
df_output = matroos.run(
    input="in_measuringstation_matroos", output="out_waterstanden_ci_postgresql_001"
)

**Ophalen waterstanden via Fews (HHNK)**

In [ ]:
# Fews HHNK waterstanden
# Vanuit HHNK: vak 6a gekoppeld aan FEWS HHNK locatie MPN-N-24
# https://fews.hhnk.nl/FewsWebServices/rest/fewspiservice/v1/timeseries?locationIds=MPN-N-24&parameterIds=WATHTE%20%5Bm%5D%5BNAP%5D%5BOW%5D
from toolbox_continu_inzicht.loads import LoadsFews

fews_hhnk = LoadsFews(data_adapter=data_adapter)
fews_hhnk.run(
    input="in_measuringstation_fews_hhnk", output="out_waterstanden_ci_postgresql_1"
)
df_out = fews_hhnk.df_out

**Ophalen waterstanden via Fews (Limburg)**

In [ ]:
# Fews Limburg waterstanden
# Vanuit WL: vak 76-1_VAK037 gekoppeld Roermond boven als die bestaat in RWS Waterwebservices
# http://109.232.44.231/FewsWebServices/rest/fewspiservice/v1/timeseries?filterId=App_Maas&locationIds=Maas%20km%2079.00%2CMaas%20km%2080.00&parameterIds=WATHTE

# from toolbox_continu_inzicht.loads import LoadsFews
# fews_hhnk = LoadsFews(data_adapter=data_adapter)
# fews_hhnk.run(input="in_measuringstation_fews_limburg", output="out_waterstanden_ci_postgresql")
# df_out = fews_hhnk.df_out


**Ophalen waterstanden via RWS Waterwebservices**

In [ ]:
# Fews Limburg waterstanden
# Vanuit WL: vak 76-1_VAK037 gekoppeld Roermond boven als die bestaat in RWS Waterwebservices
# http://109.232.44.231/FewsWebServices/rest/fewspiservice/v1/timeseries?filterId=App_Maas&locationIds=Maas%20km%2079.00%2CMaas%20km%2080.00&parameterIds=WATHTE

from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

waterwebservices = LoadsWaterwebservicesRWS(data_adapter=data_adapter)
waterwebservices.run(
    input="in_measuringstation_fews_limburg",
    output="out_waterstanden_ci_postgresql_001",
)
df_out = waterwebservices.df_out

**van alle waterstanden alleen de waterstanden van opgegeven momenten halen**

In [ ]:
from toolbox_continu_inzicht.loads import LoadsToMoments

# - waterstanden van de meetstations per moment ophalen
# - dataframe moments toevoegen aan adapter voor vervolg functies

load_moments = LoadsToMoments(data_adapter=data_adapter)
load_moments.run(
    input="in_measuringstation_data_table", output="df_moment_waterstanden"
)

if load_moments.df_out is not None:
    df_moments = load_moments.df_out.reset_index(drop=False)
    df_moments["date_time"] = df_moments["date_time"].astype(object)

    # dataframe moments toevoegen aan adapter
    data_adapter.set_dataframe_adapter("df_moment_waterstanden", df_moments)

    # print(df_moments[["hours","measurement_location_id","value_type"]])

**Status bepalen voor de meetstations**

In [ ]:
from toolbox_continu_inzicht.loads import LoadsClassify

# - via opgegeven waterstandsgrenzen wordt per meetstation de status bepaald

loads_classify = LoadsClassify(data_adapter=data_adapter)
loads_classify.run(
    input=["in_measuringstation_conditions_table", "df_moment_waterstanden"],
    output="out_measuringstation_states_table",
)

**Waterstanden bepalen voor de dijkvakken**

In [ ]:
from toolbox_continu_inzicht.sections import SectionsLoads

# - er wordt via de bovenliggende en onderliggende meetstation
#   en de fractie per meetstation (opgegeven in 'section fraction')
#   bepaald wat de maatgevende waterstand voor de dijkvak is.

if df_moments is not None:
    sections_loads = SectionsLoads(data_adapter=data_adapter)
    sections_loads.run(
        input=[
            "in_section_table",
            "df_moment_waterstanden",
            "in_sectionfractions_table",
        ],
        output="out_waterstanden_section_ci_postgresql",
    )

**Faalkans bepalen voor de dijkvakken**

In [ ]:
from toolbox_continu_inzicht.sections import SectionsTechnicalFailureprobability

sections_failureprobability = SectionsTechnicalFailureprobability(
    data_adapter=data_adapter
)
sections_failureprobability.run(
    input=["in_fragilitycurves_table", "in_section_load_from_data_table"],
    output="out_section_technical_failure_probability_data",
)

In [ ]:
from toolbox_continu_inzicht.sections import SectionsFailureprobability

sections_failureprobability = SectionsFailureprobability(data_adapter=data_adapter)
sections_failureprobability.run(
    input=["in_fragilitycurves_table", "in_section_load_from_data_table"],
    output="out_section_failure_probability_data",
)

In [ ]:
from toolbox_continu_inzicht.sections import SectionsClassify

sections_classify = SectionsClassify(data_adapter=data_adapter)
sections_classify.run(
    input=["in_section_conditions", "in_section_data_failure_probability"],
    output="out_section_states",
)

**Beeindig de berekening**

In [ ]:
from toolbox_continu_inzicht.helpers import calculation_end

calculation_end(data_adapter=data_adapter, output="calculation_end_config")